In [2]:
import pandas as pd
import numpy as np
from IPython.display import display, HTML
import json

INPUT_DIR = "/Users/ashwins/Scripts/chatbot/"

from keras.models import Model, load_model
from keras.layers import Input, Dense, LSTM, Embedding, TimeDistributed
import pickle
# import nltk
import itertools
import collections

from keras.preprocessing import sequence
from keras.models import model_from_json
from keras.preprocessing import sequence
from keras.utils import np_utils

import spacy
nlp = spacy.load('en')
# from keras.utils import preprocessing
# from model.textGenModel import TextGenModel

Using TensorFlow backend.
/Library/Anaconda3/anaconda3/envs/env1/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: compiletime version 3.6 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.5
  return f(*args, **kwds)


In [3]:
import keras
from keras import backend as K
from keras.models import Sequential
from keras.layers import Input, LSTM, RepeatVector, Bidirectional, Dropout
from keras.layers.core import Flatten, Dense, Dropout, Lambda
from keras.optimizers import SGD, RMSprop, Adam
from keras import objectives

In [4]:
# constant token and params for our models
START_TOKEN = "EOS"
END_TOKEN = "EOS"
UNKNOWN_TOKEN = "UNK"
PADDING_TOKEN = "PAD"

In [5]:
# vocabulary_size = 22285
sent_max_len = 20

# hidden_size = 512
hidden_size = 128
embedding_size = 128
batch_size = 64
stateful = False

In [6]:
import load_train_data
import models.load_ae_model as ae
import importlib

In [24]:
import models.load_ae_model_regsn as reg
import models.load_ae_model_td as td
import models.load_ae_model_td_regsn as tdreg

In [8]:
importlib.reload(load_train_data)
encoder_input_data, decoder_input_data, decoder_target_data = load_train_data.load_data()



df_lines :                                                   dialogue     l_no
0                                            They do not!    L1045
1                                             They do to!    L1044
2                                              I hope so.     L985
3                                               She okay?     L984
4                                               Let's go.     L925
5                                                     Wow     L924
6          Okay -- you're gonna need to learn how to lie.     L872
7                                                      No     L871
8       I'm kidding.  You know how sometimes you just ...     L870
9                        Like my fear of wearing pastels?     L869
10                                        The "real you".     L868
11                                       What good stuff?     L867
12      I figured you'd get to the good stuff eventually.     L866
13      Thank God!  If I had to hear one more st



decoder_input_data.shape :  (38080, 20)


decoder_input_data[:5] :  [[  157    66   207   306    24   114   305     9  1159   115    48  5178
    142     8 16665 24256     0     0     0     0]
 [   89     4   204 18786   732    21     6  2874   318   375  1399     0
      0     0     0     0     0     0     0     0]
 [  197     9   126     0     0     0     0     0     0     0     0     0
      0     0     0     0     0     0     0     0]
 [  749  8450  6992     0     0     0     0     0     0     0     0     0
      0     0     0     0     0     0     0     0]
 [   79   958     6   185    39  3630   110  1313   710     0     0     0
      0     0     0     0     0     0     0     0]]


length :  503


decoder_target_data.shape :  (38080, 20, 1)


decoder_target_data[:5] :  [[[   66]
  [  207]
  [  306]
  [   24]
  [  114]
  [  305]
  [    9]
  [ 1159]
  [  115]
  [   48]
  [ 5178]
  [  142]
  [    8]
  [16665]
  [24256]
  [  157]
  [    0]
  [    0]
  [    0]
  [    0]]

 [[    4]
 

In [9]:
print(encoder_input_data.shape)
print(decoder_input_data.shape)
print(decoder_target_data.shape)
print(encoder_input_data[:1])
print(decoder_input_data[:1])
print(decoder_target_data[:1])

(38080, 20)
(38080, 20)
(38080, 20, 1)
[[  70    4 2074  318    8    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0]]
[[  157    66   207   306    24   114   305     9  1159   115    48  5178
    142     8 16665 24256     0     0     0     0]]
[[[   66]
  [  207]
  [  306]
  [   24]
  [  114]
  [  305]
  [    9]
  [ 1159]
  [  115]
  [   48]
  [ 5178]
  [  142]
  [    8]
  [16665]
  [24256]
  [  157]
  [    0]
  [    0]
  [    0]
  [    0]]]


In [14]:
vocabulary_size = 50003

importlib.reload(ae)
auto_encoder, encoder = ae.load_model()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
enc_input (InputLayer)       (None, 1)                 0         
_________________________________________________________________
shared_emb (Embedding)       (None, 1, 32)             1600096   
_________________________________________________________________
enc_lstm (LSTM)              [(None, 32), (None, 32),  8320      
Total params: 1,608,416
Trainable params: 1,608,416
Non-trainable params: 0
_________________________________________________________________
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
dec_input (InputLayer)          (None, 1)            0                                            
__________________________________________________________________________________________________


In [17]:
importlib.reload(reg)
auto_encoder_r, encoder_r = reg.load_model()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
enc_input (InputLayer)       (None, 20)                0         
_________________________________________________________________
shared_emb (Embedding)       (None, 20, 32)            1600096   
_________________________________________________________________
enc_lstm (LSTM)              [(None, 32), (None, 32),  8320      
Total params: 1,608,416
Trainable params: 1,608,416
Non-trainable params: 0
_________________________________________________________________
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
dec_input (InputLayer)          (None, 20)           0                                            
__________________________________________________________________________________________________


In [22]:
importlib.reload(td)
auto_encoder_td, encoder_td = td.load_model()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
enc_input (InputLayer)       (None, 20)                0         
_________________________________________________________________
shared_emb (Embedding)       (None, 20, 32)            1600096   
_________________________________________________________________
enc_lstm (LSTM)              [(None, 32), (None, 32),  8320      
Total params: 1,608,416
Trainable params: 1,608,416
Non-trainable params: 0
_________________________________________________________________
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
dec_input (InputLayer)          (None, 20)           0                                            
__________________________________________________________________________________________________


In [25]:
importlib.reload(tdreg)
auto_encoder_td_r, encoder_td_r = tdreg.load_model()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
enc_input (InputLayer)       (None, 20)                0         
_________________________________________________________________
shared_emb (Embedding)       (None, 20, 32)            1600096   
_________________________________________________________________
enc_lstm (LSTM)              [(None, 32), (None, 32),  8320      
Total params: 1,608,416
Trainable params: 1,608,416
Non-trainable params: 0
_________________________________________________________________
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
dec_input (InputLayer)          (None, 20)           0                                            
__________________________________________________________________________________________________


In [23]:
import os

# model.compile(optimizer='rmsprop', loss='sparse_categorical_crossentropy')
opt = Adam(lr=0.001)
auto_encoder_td.compile(loss='sparse_categorical_crossentropy',
              optimizer=opt)

model_json = auto_encoder_td.to_json()
with open("full_model_td_new_arch", "w") as f:
    f.write(json.dumps(json.loads((model_json))))

# Note that `decoder_target_data` needs to be one-hot encoded,
# rather than sequences of integers like `decoder_input_data`!
for i in range(4):
    
        
#     if i==8:
#         opt = Adam(lr=0.00001)
#         model.compile(loss='sparse_categorical_crossentropy', optimizer=opt)
        
    model_name = 'full_model_td_new_epoch_' + str(i) + '.h5'
    auto_encoder_td.save(model_name)
    print('saved model')
    
    enc_name = 'enc_model_td_new_epoch_' + str(i) + '.h5'
    encoder_td.save(enc_name)
    print('saved enc model')
    weights_path = ''.join(['enc_model_td_new_weights_', 
                        "base_voc_{}_epoch_{}.hdf5".format(vocabulary_size, 
                                                          i)])
    encoder_td.save_weights(weights_path)
    print('saved enc weights')
    # export model weights
    weights_path = ''.join(['full_model_td_new_weights_', 
                        "base_voc_{}_epoch_{}.hdf5".format(vocabulary_size, 
                                                          i)])
    auto_encoder_td.save_weights(weights_path)
    print('saved weights')
    
    if i==3:
        opt = Adam(lr=0.0001)
        auto_encoder_td.compile(loss='sparse_categorical_crossentropy',
              optimizer=opt)
        
    auto_encoder_td.fit([encoder_input_data[:5000], decoder_input_data[:5000]], decoder_target_data[:5000], batch_size=batch_size, epochs=1, validation_split=0.2)
    

/Library/Anaconda3/anaconda3/envs/env1/lib/python3.5/site-packages/keras/engine/network.py:888: UserWarning: Layer dec_lstm was passed non-serializable keyword arguments: {'initial_state': [<tf.Tensor 'enc_lstm_4/while/Exit_3:0' shape=(?, 32) dtype=float32>, <tf.Tensor 'enc_lstm_4/while/Exit_4:0' shape=(?, 32) dtype=float32>]}. They will not be included in the serialized model (and thus will be missing at deserialization time).
  '. They will not be included '


saved model
saved enc model
saved enc weights
saved weights
Train on 4000 samples, validate on 1000 samples
Epoch 1/1
4000/4000 [==============================] - 214s 53ms/step - loss: 10.3461 - val_loss: 8.7691
saved model
saved enc model
saved enc weights
saved weights
Train on 4000 samples, validate on 1000 samples
Epoch 1/1
4000/4000 [==============================] - 223s 56ms/step - loss: 7.2678 - val_loss: 7.0338
saved model
saved enc model
saved enc weights
saved weights
Train on 4000 samples, validate on 1000 samples
Epoch 1/1
4000/4000 [==============================] - 222s 56ms/step - loss: 6.6488 - val_loss: 6.9923
saved model
saved enc model
saved enc weights
saved weights
Train on 4000 samples, validate on 1000 samples
Epoch 1/1
4000/4000 [==============================] - 197s 49ms/step - loss: 6.5862 - val_loss: 6.9898


In [29]:
import os

# model.compile(optimizer='rmsprop', loss='sparse_categorical_crossentropy')
opt = Adam(lr=0.001)
auto_encoder_r.compile(loss='mean_squared_error',
              optimizer=opt)

model_json = auto_encoder_r.to_json()
with open("full_model_new_arch", "w") as f:
    f.write(json.dumps(json.loads((model_json))))

# Note that `decoder_target_data` needs to be one-hot encoded,
# rather than sequences of integers like `decoder_input_data`!
for i in range(4):
    
        
#     if i==8:
#         opt = Adam(lr=0.00001)
#         model.compile(loss='sparse_categorical_crossentropy', optimizer=opt)
        
    model_name = 'full_model_r_new_epoch_' + str(i) + '.h5'
    auto_encoder_r.save(model_name)
    print('saved model')
    
    enc_name = 'enc_model_r_new_epoch_' + str(i) + '.h5'
    encoder_r.save(enc_name)
    print('saved enc model')
    weights_path = ''.join(['enc_model_r_new_weights_', 
                        "base_voc_{}_epoch_{}.hdf5".format(vocabulary_size, 
                                                          i)])
    encoder_r.save_weights(weights_path)
    print('saved enc weights')
    # export model weights
    weights_path = ''.join(['full_model_r_new_weights_', 
                        "base_voc_{}_epoch_{}.hdf5".format(vocabulary_size, 
                                                          i)])
    auto_encoder_r.save_weights(weights_path)
    print('saved weights')

    if i==3:
        break
        opt = Adam(lr=0.0001)
        auto_encoder_r.compile(loss='mean_squared_error',
              optimizer=opt)
    
    auto_encoder_r.fit([encoder_input_data[:5000], decoder_input_data[:5000]], decoder_target_data[:5000], batch_size=batch_size, epochs=1, validation_split=0.2)
    

/Library/Anaconda3/anaconda3/envs/env1/lib/python3.5/site-packages/keras/engine/network.py:888: UserWarning: Layer dec_lstm was passed non-serializable keyword arguments: {'initial_state': [<tf.Tensor 'enc_lstm_9/while/Exit_3:0' shape=(?, 32) dtype=float32>, <tf.Tensor 'enc_lstm_9/while/Exit_4:0' shape=(?, 32) dtype=float32>]}. They will not be included in the serialized model (and thus will be missing at deserialization time).
  '. They will not be included '


saved model
saved enc model
saved enc weights
saved weights
Train on 4000 samples, validate on 1000 samples
Epoch 1/1
4000/4000 [==============================] - 9s 2ms/step - loss: 69468240.3200 - val_loss: 71810720.0960
saved model
saved enc model
saved enc weights
saved weights
Train on 4000 samples, validate on 1000 samples
Epoch 1/1
4000/4000 [==============================] - 6s 2ms/step - loss: 69388372.9920 - val_loss: 71799783.6160
saved model
saved enc model
saved enc weights
saved weights
Train on 4000 samples, validate on 1000 samples
Epoch 1/1
4000/4000 [==============================] - 5s 1ms/step - loss: 69974015.1040 - val_loss: 71788903.4880
saved model
saved enc model
saved enc weights
saved weights
